In [4]:
import pandas as pd
pd.options.display.float_format = '{:,.4f}'.format
pd.options.display.max_rows = 100
pd.options.display.max_columns = 40
import numpy as np
import os, shutil
from tqdm import tqdm

# Warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# Web Scraping
from selenium import webdriver
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.keys import Keys
from time import sleep
from bs4 import BeautifulSoup
from tqdm import tqdm
options = webdriver.ChromeOptions()

options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-gpu")

options.add_argument("window-size=1920x1080")
options.add_argument("lang=ko_KR")
options.add_argument("user-agent=Chrome/77.0.3865.90")

from urllib.request import urlopen
from urllib.parse import urlencode, unquote, quote_plus
import urllib, requests, json
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import numpy as np
import os, re

# cf.) functions

In [5]:
# 기존 주소, 입력받은 주소를 변환한 결과, 건물명, 주소 폐지 여부를 txt파일로 저장한 뒤에 이를 읽어들여 return
# 도로명주소 -> 지번주소 or 지번주소 -> 도로명주소(양방향 가능)
def transform_address(address_ls, filepath, output):
    # output 저장할 파일 생성
    with open(output, "w") as f:
        f.write("지번주소;도로명주소;건물명;폐지여부\n")
        
    # Web Scraping
    url = "https://www.juso.go.kr/openIndexPage.do"
    driver = webdriver.Chrome(executable_path = 'C:/Program Files/chromedriver/chromedriver.exe',options = options)

    for address in tqdm(address_ls):
        driver.get(url)
        selected_tag_a=driver.find_element_by_css_selector('input#inputSearchAddr')
        selected_tag_a.click()
        selected_tag_a.send_keys(address) 
        selected_tag_a.find_element_by_xpath('//*[@id="AKCFrm"]/fieldset/div/button').send_keys((Keys.ENTER))
        source=driver.page_source
        soup=BeautifulSoup(source,'lxml') 
        try:
            new_address = soup.select('#rnAddr1')[0].get('value').split('(')[0]
            abo = "존재"
        except IndexError: # 검색 결과가 없는 경우
            print(f"{address}은(는) 폐지된 주소 정보입니다.")
            driver.find_element_by_css_selector('input#ckAblYn').click()
            driver.find_element_by_css_selector('button#searchButton').click()
            new_address = soup.select('#rnAddr1')[0].get('value').split('(')[0]
            abo = "폐지"
        except TimeoutException as ex:
            print(ex.Message)
            webDriver.navigate().refresh()
        old_address = soup.select('#lndnAddr1')[0].get('value')
        bdNm = soup.select('#bdNm1')[0].get('value')
        
        # output txt파일에 저장
        if address == old_address:
            old_address = address
        elif address == new_address:
            new_address = address
        with open(os.path.join(filepath, output), "a") as f:
            f.write(f"{old_address};{new_address};{bdNm};{abo}\n")
    # txt파일 불러오기
    trans_address = pd.read_csv(os.path.join(filepath, output), sep=";",encoding ="cp949")
    return trans_address


# 건축물대장 API 호출
def call_API(params, API_KEY):
    API_KEY = API_KEY
    API_URL = 'http://apis.data.go.kr/1613000/BldRgstService_v2/getBrTitleInfo'
    sigunguCd, bjdongCd, bun, ji = map(str, params)
    queryParams = '?' + urlencode({quote_plus('serviceKey') : API_KEY, 
                                   quote_plus('sigunguCd') : sigunguCd, 
                                   quote_plus('bjdongCd') : bjdongCd, 
                                   quote_plus('platGbCd') : "0", 
                                   quote_plus('bun') : bun, 
                                   quote_plus('ji') : ji, 
                                   quote_plus('numOfRows') : '10',
                                   quote_plus('pageNo') : '1' })
    response = requests.get(API_URL+queryParams).text.encode('utf-8')
    xmlobj = BeautifulSoup(response, 'html.parser')
    return xmlobj
    
# 입력받은 tag를 기준으로 API 호출 결과 parsing    
def get_data(xmlobj, tag):
    try:
        result = xmlobj.findAll(tag)[0].text
    except:
        result = re.findall("\d+", str(xmlobj.findAll(tag)[0]))[0]
    return result

# 도로명주소를 입력받아 시군구코드, 법정동코드, 지번 추출
def get_params(trans_address, new_address):
    old_address = trans_address.loc[trans_address.도로명주소==new_address,"지번주소"].values[0]
    bjdongNm = re.findall('.*?동\s|.*?구\s|.*?리\s|.*?가\s', old_address)[0].strip()
    try:
        code = bupjeongdong.loc[bupjeongdong.법정동명==bjdongNm,'법정동코드'].unique()[0]
    except:
        print(f"{new_address}의 법정동명이 없습니다.\nAPI 호출이 불가능합니다.")
        params = []
    else: 
        sigunguCd = str(code)[:5] # 시군구코드
        bjdongCd = str(code)[5:] # 법정동코드
        # 지번 추출
        if "-" in old_address: 
            # 지번 주소에 '-'이 있는 경우
            # ex) 336-1 -> bun = 0336, ji = 0001
            bun_ls = old_address.split("-")
            bun = re.findall("\d+", bun_ls[0])[0].zfill(4)
            ji = re.findall("\d+", bun_ls[1])[0].zfill(4)
        else:
            # 지번 주소에 '-'이 없는 경우
            # ex) 695 -> bun = 0695, ji = 0000
            bun = re.findall("\d+", old_address)[0].zfill(4)
            ji = "0000"
        params = [sigunguCd, bjdongCd, bun, ji]       
    return params
            
# 도로명주소를 입력받아 해당 건축물의 정보를 데이터프레임 형태로 return
def get_one_building(trans_address, new_address, k_var, v_var, API_KEY):
    old_address = trans_address.loc[trans_address.도로명주소==new_address,"지번주소"].values[0]
    bjdongNm = re.findall('.*?동\s|.*?구\s|.*?리\s|.*?가\s', old_address)[0].strip()
    # OpenAPI 호출
    API_params = get_params(trans_address, new_address)
    if not API_params: # 법정동코드를 찾지 못해 API 호출이 불가능한 경우
        df = pd.DataFrame({k: np.NaN for k in k_var}, index=[0])
    else: # API 호출
        xmlobj = call_API(API_params, API_KEY)      
        res_dic = {k: '' for k in k_var}
        tag_list = list(map(lambda x: x.lower(), v_var)) # parsing에 필요한 변수명 tag
        try:
            data = [get_data(xmlobj, tag) for tag in tag_list]
            for idx, i in  enumerate(res_dic.keys()):
                res_dic[i] = [data[idx]]
            df = pd.DataFrame(res_dic) 
        except:
            print(f"{new_address}의 검색 결과가 없습니다.")
            df = pd.DataFrame({k: np.NaN for k in k_var}, index=[0])

    df['도로명주소'] = new_address
    df['지번주소'] = old_address

    return df

# get_archarea를 반복 호출해 하나의 데이터프레임을 return
def get_building(trans_address, address, k_var, v_var, API_KEY):
    building = pd.DataFrame(columns=k_var)
    for i in tqdm(address):
        res = get_one_building(trans_address,i,k_var,v_var, API_KEY)
        building = pd.concat([building,res])
    building.reset_index(inplace=True)
    del building['index']
    return building  

# 데이콘 데이터셋 - train.csv, test.csv
- new_address_tr - train 도로명주소
- new_address_tst - test 도로명주소

In [10]:
filepath = '../ProcessedData'
train = pd.read_csv(os.path.join(filepath,'merged_train.csv'))
print(train.단지코드.nunique())
print(train.도로명주소.nunique())

421
421


In [12]:
filepath = '../ProcessedData'
test = pd.read_csv(os.path.join(filepath,'merged_test.csv'))
print(test.단지코드.nunique())
print(test.도로명주소.nunique())

150
150


In [8]:
new_address_tr = list(train.도로명주소.unique())
new_address_tst = list(test.도로명주소.unique())
print(train.도로명주소.nunique()+test.도로명주소.nunique())
print(len(list(train.도로명주소.unique())+list(test.도로명주소.unique())))

571
571


In [11]:
# 단지명 기준으로 train, test 중복 처리
# 서로 다른 지역의 동일 단지명인 Case
dup_list = train.loc[train.단지명.isin(test.단지명.unique()),'단지명'].unique()
print(*dup_list, sep="\n")
danji_tr = {}
danji_tst = {}
for i in dup_list:
    danji_tr[i] = train.loc[train.단지명==i,'단지코드'].unique()[0]
    danji_tst[i] = test.loc[test.단지명==i,'단지코드'].unique()[0]
danji_tr, danji_tst
print(danji_tr)
print(danji_tst)

휴먼시아
송정주공아파트
{'휴먼시아': 'C1379', '송정주공아파트': 'C1569'}
{'휴먼시아': 'C2423', '송정주공아파트': 'C2144'}


In [12]:
# 도로명주소 기준으로 train, test 중복 처리
# 동일 지역의 서로 다른 단지인 Case
dup_list = train.loc[train.도로명주소.isin(test.도로명주소.unique()),'도로명주소'].unique()
print(*dup_list, sep="\n")
danji_tr = {}
danji_tst = {}
for i in dup_list:
    danji_tr[i] = train.loc[train.도로명주소==i,'단지코드'].unique()[0]
    danji_tst[i] = test.loc[test.도로명주소==i,'단지코드'].unique()[0]
danji_tr, danji_tst
print(danji_tr)
print(danji_tst)

경기도 용인시 기흥구 흥덕1로79번길 7
{'경기도 용인시 기흥구 흥덕1로79번길 7': 'C1036'}
{'경기도 용인시 기흥구 흥덕1로79번길 7': 'C2675'}


# 공공임대주택 단지정보 - housing.csv

In [13]:
filepath = "../External"
housing = pd.read_csv(os.path.join(filepath,'external_data.csv'))

In [14]:
print(housing.columns)
housing.columns = ['기관명', '광역시도코드', '지역', '시군구코드', '시군구명', '단지명', '도로명주소', '총세대수',
       '공급유형', '형명', '전용면적', '공용면적', '임대건물구분', '주차수', '기본임대보증금', '기본월임대료',
       '기본전환보증금']
print(housing.columns)

Index(['기관명', '광역시도코드', '광역시도명', '시군구코드', '시군구명', '단지명', '도로명주소', '세대수',
       '공급유형명', '형명', '공급전용면적', '공급공용면적', '주택유형명', '주차수', '기본임대보증금', '기본월임대료',
       '기본전환보증금'],
      dtype='object')
Index(['기관명', '광역시도코드', '지역', '시군구코드', '시군구명', '단지명', '도로명주소', '총세대수', '공급유형',
       '형명', '전용면적', '공용면적', '임대건물구분', '주차수', '기본임대보증금', '기본월임대료', '기본전환보증금'],
      dtype='object')


# 도로명주소 -> 지번주소 변환

In [16]:
filepath = '../ProcessedData'
output = 'trans_address_tr.txt'
# trans_address_tr = transform_address(new_address_tr, filepath, output)
trans_address_tr = pd.read_csv(os.path.join(filepath,output),sep=";",encoding="cp949")
trans_address_tr 

,지번주소,도로명주소,건물명,폐지여부
0,경상북도 포항시 북구 양덕동 336-1,경상북도 포항시 북구 장량중앙로 17,장량휴먼시아,존재
1,경상남도 통영시 미수동 695,경상남도 통영시 미우지해안로 107,미수휴먼시아,존재
2,대전광역시 유성구 상대동 465,대전광역시 유성구 상대로 40,도안휴먼시아4단지,존재
3,경기도 남양주시 호평동 712,경기도 남양주시 늘을3로 8,주공Humansia,존재
4,전라북도 익산시 오산면 장신리 1130,전라북도 익산시 오산면 선화로4길 66,장신휴먼시아1단지,존재
...,...,...,...,...
416,제주특별자치도 서귀포시 서홍동 355-1,제주특별자치도 서귀포시 장수로 115,서귀포서홍 행복주택,존재
417,강원도 춘천시 우두동 1088,강원도 춘천시 우두1길 40,춘천우두엘에이치천년나무3단지,존재
418,부산광역시 남구 용호동 376-9,부산광역시 남구 동명로 183,NaN,존재
419,충청북도 제천시 고암동 1273,충청북도 제천시 용두천로40길 28,NaN,존재


In [17]:
filepath = '../ProcessedData'
output = 'trans_address_tst.txt'
# trans_address_tst = transform_address(new_address_tst, filepath, output)
trans_address_tst = pd.read_csv(os.path.join(filepath,output),sep=";",encoding="cp949")
trans_address_tst

,지번주소,도로명주소,건물명,폐지여부
0,경기도 광명시 소하동 1391,경기도 광명시 서원로 25,광명역세권휴먼시아,존재
1,경기도 광명시 소하동 1373,경기도 광명시 성채안로 26,광명역세권휴먼시아,존재
2,부산광역시 동래구 안락동 1256,부산광역시 동래구 충렬대로447번길 38,부산안락4 휴먼시아,존재
3,전라북도 익산시 오산면 장신리 1126,전라북도 익산시 오산면 선화로4길 14,장신휴먼시아3단지,존재
4,경기도 성남시 분당구 삼평동 728,경기도 성남시 분당구 동판교로 212,봇들마을6단지아파트,존재
...,...,...,...,...
145,제주특별자치도 서귀포시 성산읍 고성리 1141-1,제주특별자치도 서귀포시 성산읍 고성동서로 33,서귀포성산엘에이치아파트,존재
146,충청북도 충주시 호암동 1066,충청북도 충주시 호암수청2로 56,충주호암1단지,존재
147,강원도 화천군 화천읍 신읍리 1818,강원도 화천군 화천읍 산천어길 196,공공실버주택,존재
148,경상남도 창원시 마산합포구 가포동 58-2,경상남도 창원시 마산합포구 가포본동5길 37,창원가포 엘에이치아파트 1단지,존재


# 법정동 - bupjeongdong.csv

In [21]:
filepath = "./"
bupjeongdong = pd.read_csv(os.path.join(filepath, "bupjeongdong.csv"),"\t",encoding="cp949")
bupjeongdong

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
46287,5013032022,제주특별자치도 서귀포시 표선면 하천리,존재
46288,5013032023,제주특별자치도 서귀포시 표선면 성읍리,존재
46289,5013032024,제주특별자치도 서귀포시 표선면 가시리,존재
46290,5013032025,제주특별자치도 서귀포시 표선면 세화리,존재


# 건축물대장정보 - building.csv

In [2]:
API_KEY = 'aaJUNB0G9RL42RIOAUXocxGfIZRGIP9cxVQ+7+IvE8xiQBA6rhn6AAMqQ81eOudK/8Hb6jfX48cDA2BtbsGt4g=='

In [26]:
# k_var = ["관리건축물대장PK","건물명","연면적","옥내기계식대수","옥외기계식대수","옥내자주식대수","옥외자주식대수"]
# v_var = ["mgmBldrgstPk","bldNm","totArea","indrMechUtcnt","oudrMechUtcnt","indrAutoUtcnt","oudrAutoUtcnt"]
# building_tr = get_building(trans_address_tr, new_address_tr,k_var,v_var,API_KEY=API_KEY)

  8%|██████▎                                                                          | 33/421 [00:41<08:47,  1.36s/it]

경상남도 창원시 마산회원구 내서읍 삼계2길 24의 검색 결과가 없습니다.


  8%|██████▌                                                                          | 34/421 [00:41<07:23,  1.15s/it]

경상남도 창원시 마산회원구 내서읍 상곡로 49의 검색 결과가 없습니다.


 19%|███████████████▌                                                                 | 81/421 [01:25<03:47,  1.50it/s]

...

 98%|██████████████████████████████████████████████████████████████████████████████  | 411/421 [07:42<00:02,  3.59it/s]

충청북도 청주시 상당구 월운로 146의 검색 결과가 없습니다.


 99%|██████████████████████████████████████████████████████████████████████████████▊ | 415/421 [07:44<00:03,  1.67it/s]

충청북도 청주시 상당구 호미로 137의 검색 결과가 없습니다.


100%|████████████████████████████████████████████████████████████████████████████████| 421/421 [07:59<00:00,  1.14s/it]


In [38]:
# building_tr.옥내기계식대수=building_tr.옥내기계식대수.astype("float")
# building_tr.옥외기계식대수=building_tr.옥외기계식대수.astype("float")
# building_tr.옥내자주식대수=building_tr.옥내자주식대수.astype("float")
# building_tr.옥외자주식대수=building_tr.옥외자주식대수.astype("float")

In [43]:
building_tr.loc[building_tr.옥내자주식대수>0,]

,관리건축물대장PK,건물명,연면적,옥내기계식대수,옥외기계식대수,옥내자주식대수,옥외자주식대수,도로명주소,지번주소
41,26320-2107,금곡7단지주공아파트,563.59,0.0000,0.0000,80.0000,215.0000,부산광역시 북구 금곡대로550번길 23,부산광역시 북구 금곡동 811
224,48129-2000017418,자은3주공아파트,13.977,0.0000,0.0000,218.0000,393.0000,경상남도 창원시 진해구 진해대로975번길 23,경상남도 창원시 진해구 자은동 760-1
420,42760-100179404,평창하진부주공아파트,24661.238,0.0000,0.0000,139.0000,32.0000,강원도 평창군 진부면 경강로 3587,강원도 평창군 진부면 하진부리 115


In [117]:
# k_var = ["관리건축물대장PK","건물명","대지면적","건축면적","연면적","용적률산정연면적","용적률","세대수","가구수"]
# v_var = ["mgmBldrgstPk","bldNm","platArea","archArea","totArea","vlRatEstmTotArea","vlRat","hhldCnt","fmlyCnt"]
# building_tr = get_building(trans_address_tr, new_address_tr,k_var,v_var,API_KEY=API_KEY)

  8%|██████▎                                                                          | 33/421 [00:06<01:08,  5.68it/s]

경상남도 창원시 마산회원구 내서읍 삼계2길 24의 검색 결과가 없습니다.
경상남도 창원시 마산회원구 내서읍 상곡로 49의 검색 결과가 없습니다.


 19%|███████████████▊                                                                 | 82/421 [00:14<00:54,  6.25it/s]

경기도 부천시 양지로 20의 검색 결과가 없습니다.


 34%|███████████████████████████▎                                                    | 144/421 [00:24<00:40,  6.82it/s]

...

 98%|██████████████████████████████████████████████████████████████████████████████▎ | 412/421 [01:06<00:01,  6.80it/s]

충청북도 청주시 상당구 월운로 146의 검색 결과가 없습니다.


 99%|███████████████████████████████████████████████████████████████████████████████ | 416/421 [01:07<00:00,  7.02it/s]

충청북도 청주시 상당구 호미로 137의 검색 결과가 없습니다.


100%|████████████████████████████████████████████████████████████████████████████████| 421/421 [01:08<00:00,  6.18it/s]


In [118]:
# k_var = ["관리건축물대장PK","건물명","대지면적","건축면적","연면적","용적률산정연면적","용적률","세대수","가구수"]
# v_var = ["mgmBldrgstPk","bldNm","platArea","archArea","totArea","vlRatEstmTotArea","vlRat","hhldCnt","fmlyCnt"]
# building_tst = get_building(trans_address_tst, new_address_tst,k_var,v_var,API_KEY=API_KEY)

 15%|███████████▉                                                                     | 22/150 [00:05<00:25,  5.11it/s]

울산광역시 북구 명촌로 94의 검색 결과가 없습니다.


 43%|██████████████████████████████████▌                                              | 64/150 [00:14<00:16,  5.36it/s]

경기도 부천시 소사로 639의 검색 결과가 없습니다.


 64%|███████████████████████████████████████████████████▊                             | 96/150 [00:21<00:09,  5.99it/s]

전라북도 익산시 선화로1길 25-14의 검색 결과가 없습니다.


 80%|████████████████████████████████████████████████████████████████                | 120/150 [00:27<00:05,  5.30it/s]

경기도 안성시 아양로 73의 검색 결과가 없습니다.


100%|████████████████████████████████████████████████████████████████████████████████| 150/150 [00:33<00:00,  4.51it/s]

경상남도 창원시 마산합포구 가포본동5길 37의 검색 결과가 없습니다.


In [ ]:
k_var = ["관리건축물대장PK","건물명","연면적","옥내기계식대수","옥외기계식대수","옥내자주식대수","옥외자주식대수"]
v_var = ["mgmBldrgstPk","bldNm","totArea","indrMechUtcnt","oudrMechUtcnt","	indrAutoUtcnt","oudrAutoUtcnt"]
building_tst = get_building(trans_address_tst, new_address_tst,k_var,v_var,API_KEY=API_KEY)

In [132]:
filepath = "../External3"
building_tr.to_csv(os.path.join(filepath,"building_tr.csv"),index=False)
building_tst.to_csv(os.path.join(filepath,"building_tst.csv"),index=False)

In [47]:
filepath = "../ProcessedData"
building_tr = pd.read_csv(os.path.join(filepath,"building_tr.csv"))
building_tst = pd.read_csv(os.path.join(filepath,"building_tst.csv"))
# NA 개수
pd.DataFrame({'train':building_tr.isna().sum(),
              'test':building_tst.isna().sum()})

,train,test
관리건축물대장PK,24,5
건물명,24,5
대지면적,24,5
건축면적,24,5
연면적,24,5
용적률산정연면적,24,5
용적률,24,5
세대수,24,5
가구수,24,5
도로명주소,0,0
